In [ ]:
# this file here is trying to use SSIM with SRGAN and I have not been able to make it work better then SSIM
from dlsr import *
import numpy as np
import os
from PIL import Image
from dlsr.data import DIV2K

In [ ]:
image_size = 32
batch_size = 128
training_data_size = 6400
validation_data_size = 128
num_to_predict = 10
predict_size = 192

output_path = "./output/temp"

In [ ]:
from dlsr.data.div2k import random_crop


training_data = helpers.get_training_data(
    num=training_data_size, valid_num=validation_data_size, image_size=image_size
)

callback_ds = DIV2K(type="valid").dataset(batch_size=1, random_transform=False, crop_images=True, image_size=predict_size)
callback_ds = callback_ds.map(lambda a, b: a)
callback_ds = callback_ds.take(num_to_predict)

In [ ]:
def callback(generator, i):
    directory = f"{output_path}/{i}"
    try:
        os.makedirs(directory)
    except FileExistsError:
        # directory already exists
        pass

    generator.save(f"{directory}/generator.h5")

    upscaled = upscale(generator, callback_ds, batch_size=num_to_predict)

    for i , x in enumerate(upscaled):
        im = Image.fromarray(np.uint8(x))
        im.save(os.path.join(directory, f"converted{i}.png"), format="png")

In [ ]:
try:
    os.makedirs(output_path)
except FileExistsError:
    # directory already exists
    pass

discriminator = models.discriminator(image_size=image_size)
generator = models.basic()
generator, gen_his, discriminator, dis_his = train_gan(
    discriminator=discriminator,
    generator=generator,
    training_data=training_data,
    epochs=200,
    discriminator_epochs=25,
    discriminator_patience=5,
    discriminator_batch_size=batch_size,
    discriminator_n=2e-6,
    generator_epochs=25,
    generator_loss_fn=losses.get_srgan_loss(discriminator)["srgan_ssim_loss"],
    generator_patience=5,
    generator_batch_size=batch_size,
    generator_n=2e-5,
    pre_train_epochs=200,
    pre_train=True,
    pre_train_loss_fn=losses.ssim_loss,
    epoch_start_callback=callback
)

In [ ]:
generator.save("./saved-models/srgan/generator.h5")
discriminator.save("./saved-models/srgan/discriminator.h5")

In [ ]:
helpers.plot_history(gen_his)
helpers.plot_history(dis_his)